In [61]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

In [15]:
data = pd.read_csv('cleantraining.csv')

In [43]:
UnrankedGames = pd.read_csv('cleantesting.csv')
UnrankedGames =UnrankedGames.reindex(columns=data.columns)
UnrankedX = UnrankedGames.drop(['id','bayesaverage','name','descriptions','mechanics','categories'], axis=1)

In [24]:
y=data['bayesaverage']
X=data.drop(['id','bayesaverage','name','descriptions','mechanics','categories'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3585)

##### Lasso Regresion

In [28]:
pipeLasso = Pipeline([
    ('scale', StandardScaler()),
    ('model', LassoCV())
])

In [29]:
pipeLasso.fit(X_train,y_train)

Pipeline(steps=[('scale', StandardScaler()), ('model', LassoCV())])

In [31]:
yhat=pipeLasso.predict(X_test)
np.sqrt(mean_squared_error(y_test,yhat))

0.4634214877221356

##### Random Forest

In [47]:
RFmod = RandomForestRegressor()
RFmod.fit(X_train, y_train)

RandomForestRegressor()

In [48]:
yhat=RFmod.predict(X_test)
np.sqrt(mean_squared_error(y_test,yhat))

0.4285098093850316

##### KNN

In [54]:
pipeknn = Pipeline([
    ('scale', StandardScaler()),
    ('model', KNeighborsRegressor(n_neighbors=30))
])
pipeknn.fit(X_train, y_train)

Pipeline(steps=[('scale', StandardScaler()),
                ('model', KNeighborsRegressor(n_neighbors=30))])

In [55]:
yhat=pipeknn.predict(X_test)
np.sqrt(mean_squared_error(y_test,yhat))

0.6798263631261069

May be a bit underfit

##### SVM

In [62]:
pipesvm = Pipeline([
    ('scale', StandardScaler()),
    ('model', SVR(kernel='rbf'))
])
pipesvm.fit(X_train, y_train)

Pipeline(steps=[('scale', StandardScaler()), ('model', SVR())])

In [63]:
yhat=pipesvm.predict(X_test)
np.sqrt(mean_squared_error(y_test,yhat))

0.5522500176259699

##### Create a table with Predictions

In [56]:
predRF = RFmod.predict(UnrankedX)
predKNN = pipeknn.predict(UnrankedX)

In [64]:
predLasso = pipeLasso.predict(UnrankedX)
predSVM = pipesvm.predict(UnrankedX)

In [65]:
predDF = pd.DataFrame({'Name': UnrankedGames['name'],
                       'PredLasso': predLasso,
                       'PredRF': predRF,
                       'PredKNN': predKNN,
                       'PredSVM': predSVM,
                       'OldRating': UnrankedGames['bayesaverage']})

In [ ]:
predDF['AvgPred']=predDF.loc[: , "PredLasso":"PredSVM"].mean(axis=1)

In [72]:
predDF.sort_values(by='AvgPred', ascending=False)

,Name,PredLasso,PredRF,PredKNN,PredSVM,OldRating,AvgPred
194,To Take Washington: Jubal Early's Summer 1864 ...,7.692874,7.855165,6.969739,7.142941,5.57647,7.415180
449,Vietnam: 1965-1975 (2nd edition),7.475841,7.757145,6.965150,7.142916,5.66859,7.335263
138,Legacy of Thracks: The Awakening,7.383316,7.770061,6.709580,7.180343,5.51825,7.260825
15,Wild Gardens,7.802215,7.120847,6.619307,7.435253,5.62107,7.244406
392,Periorbis,7.325933,7.732180,6.689146,7.126318,5.52659,7.218395
...,...,...,...,...,...,...,...
93,Big Brother: The Game,5.892596,5.950126,5.993004,5.848536,5.43686,5.921065
114,Rock Paper Scissors,5.868866,5.284767,6.302632,6.150663,5.43849,5.901732
424,Tilt'n Tumble,5.823522,5.990794,6.120579,5.656010,5.49995,5.897726
421,Biljard,5.607678,6.303566,6.245016,5.433014,5.51144,5.897319
